In [39]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file


client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [40]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a Responsible AI expert and going to help a industry cross-functional team evaluat AI product design plans with the goal of identifying all potential ethical issues that could cause downstream harms",
            },
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
    )
    # print(chat_completion.choices[0])
    return chat_completion.choices[0].message.content

In [41]:
worksheet = {
    "problem": "Brief the problem or challenge that can be solved by AI in simple words, highlighting its significance and potential impact on target users or stakeholders",
    "task": "Explain how AI focuses on the specific task that aims to solve the problem.",
    "data": "Describe where and how the data for training the AI system is collected and prepared in plain language, emphasizing the data preprocessing steps and any feature engineering technologies that are applied to the raw data, address any potential biases or ethical considerations related to the data.",
    "model": "Explain what AI model architecture and algorithms are being used and their respective roles in simple terms.",
    "train": "Describe how the AI model learns from the curated data, and clarify the process of how it improves its performance.",
    "test": "Explain how the AI model is evaluated and assessed for its effectiveness and accuracy, using plain words to highlight the testing process.",
    "deploy": "Describe how the AI system is deployed in practical use, emphasizing the benefits and potential impact on users or stakeholders.",
    "feedback": "Explain how feedback is gathered from users or stakeholders, and the model is monitored in interns of the generalization and drifting to improve the AI system and highlight how it helps the iteration of AI development."
}

checklist = [
    "Briefly describe the issue:",
    "Who is at risk of experiencing the issue?",
    "What is the potential downstream impact?",
    "How severe is the impact? Likert scale 1-7",
    "Any other comments?"
]

In [42]:
ai_lego_example = {
    "descriptions": {
        "problem": [
            "The developers want to create a voice-activated virtual assistant capable of recognizing speech and executing various tasks given by users."
        ],
        "task": [
            "Alexa’s will be able to handle voice recognition, natural language processing, and providing responses to user commands",
            "Alexa need to perform robustly in various ambient conditions."
        ],
        "data": [
            "A large amount of data is collected and curated. This data includes speech samples, language patterns, and various scenarios that the AI should be able to handle. The data involves different voices, accents, and potential voice commands for speech recognition, and language data for natural language processing"
        ],
        "model": [
            "Design the architecture and algorithms of the AI model, which may involve technologies like Automatic Speech Recognition (ASR), Language Modeling, and Acoustic Fingerprinting",
            "Use rule-based conversational system such as DialogueFlow with ChatGPT API"
        ],
        "train": [
            "Train the AI model using the curated data to optimize its performance in understanding and responding to user inputs",
        ],
        "test": [
            "Evaluate the AI model's performance and accuracy using different test scenarios to ensure it meets the desired requirements"
        ],
        "deploy": [
            "Integrate the trained AI model into the Amazon Alexa products (Echo, Echo Dot) for real-world use",
            "Deploy the model on cloud such as AWS"
        ],
        "feedback": [
            "Monitor the AI system's behavior and gather user feedback to identify any issues or unintended consequences. Use this feedback to improve the system through updates and patches",
            "Sampling the conversations between users and Alexa and periodically evaluate the quality."
        ],
    },
    "comments": {
        "problem": [
            ["What does \"virtual assistant\" mean?"],
        ],
        "task": [
            [],
            ["please define robustness"]
        ],
        "data": [
            [],
        ],
        "model": [
            [],
            [],
        ],
        "train": [
            [],
        ],
        "test": [
            [],
        ],
        "deploy": [
            [],
            [],
        ],
        "feedback": [
            [],
            [],
        ]  
    },
    "evaluations": {
        
    },
    "lines": [
        ["problem-0", "task-0", "data-0", "model-0", "train-0", "test-0", "deploy-0", "feedback-0"],
        ["problem-0", "task-1", "data-0", "model-1", "deploy-1", "feedback-1"],
    ]
}

In [55]:
## Task 1: Summarization
prompts_block = "\n".join([f"{stage}: {stage_prompt}" for stage, stage_prompt in worksheet.items()])

for line_idx, line in enumerate(ai_lego_example["lines"]):
    plan_block = ""
    for stage in line:
        stage_name, stage_idx = stage.split('-')
        stage_idx = int(stage_idx)
        plan_block += f"{stage_name}: {ai_lego_example['descriptions'][stage_name][stage_idx]}"
        if stage != line[-1]:
            plan_block += "\n"
    # print(plan_block, prompts_block)
    
    prompt = f"""
Your task is to generate a short summary of an AI product development plan a team have been working on.

The plan was written by the team across 8 stages following the prompts below:

Prompts:
```
{prompts_block}
```

Summarize the development plan in a cohesive paragraph, delimited by triple 
backticks, in at most 100 words. 

Development plan:
```
{plan_block}
```
"""
    
    # print(prompt)

    response = get_completion(prompt)
    print(f"Plan {line_idx}:\n", response.replace('`','').strip())

Plan 0:
 The team aims to develop a voice-activated virtual assistant, like Alexa, to recognize speech and perform tasks. They gather diverse speech and language data for training, preventing biases. The AI model uses ASR and Language Modeling for speech recognition. By training on curated data, the model improves in understanding user commands. After rigorous testing, the AI is deployed into Amazon Alexa devices for real-world use, monitoring user feedback to enhance the system iteratively and ensure ethical use.
Plan 1:
 The team is developing a voice-activated virtual assistant, Alexa, to understand and execute user commands effectively in diverse environments. Data is meticulously curated, including varied speech samples, language patterns, and scenarios, considering different voices and accents for robust performance. Deploying a rule-based conversational system like DialogueFlow with ChatGPT API on AWS, ensuring easy access. Continuous feedback through user conversations aids in 

In [54]:
## Task 2: Translate

role_from = "ml engineer"
role_to = "product manager"
stage_name, stage_idx = "model", 0
    
prompt = f"""
Your task is to rewrite the description for a stage of AI product development plan wrote by {role_from} so that it is understandable by a {role_to}.

The stage and the prompt of the description are:
```
{stage_name}: {worksheet[stage_name]}
```

Rewrite the description in a cohesive paragraph, delimited by triple 
backticks, in at most 500 words. 

Description:
```
{ai_lego_example["descriptions"][stage_name][stage_idx]}
```
"""

response = get_completion(prompt)
print(f"Orignal description from {role_from}:\n", ai_lego_example["descriptions"][stage_name][stage_idx])
print()
print(f"Rewritten description for {role_to}:\n", response.replace('`','').strip())

Orignal description from ml engineer:
 Design the architecture and algorithms of the AI model, which may involve technologies like Automatic Speech Recognition (ASR), Language Modeling, and Acoustic Fingerprinting

Rewritten description for product manager:
 In this stage, we will determine the structure and algorithms that the AI model will utilize. We will consider using tools like Automatic Speech Recognition (ASR), Language Modeling, and Acoustic Fingerprinting to enhance the AI model's capabilities. Each of these technologies plays a specific role in enabling the model to process and understand input data effectively. By designing the architecture and selecting the appropriate algorithms, we aim to optimize the AI model's performance and accuracy in tasks such as speech recognition and language processing.


In [60]:
## Task 3: Refine / Suggestions

stage_name, stage_idx = "model", 0
    
prompt = f"""
Your task is to provide suggestion on how to improve the quality of a description of a stage of AI product development plan based on its prompt.

The stage and the prompt which you can use for criterias of the description are:
```
{stage_name}: {worksheet[stage_name]}
```

Generate the suggestions in a list where each line is one suggestion starting with a number enumerator. Each suggestion should be at most 50 words.

Description:
```
{ai_lego_example["descriptions"][stage_name][stage_idx]}
```
"""

response = get_completion(prompt)
print(f"Original:\n", ai_lego_example["descriptions"][stage_name][stage_idx])
print()
print(f"Suggestions:\n", response)

Original:
 Design the architecture and algorithms of the AI model, which may involve technologies like Automatic Speech Recognition (ASR), Language Modeling, and Acoustic Fingerprinting

Suggestions:
 1. Provide specific details on how ASR, Language Modeling, and Acoustic Fingerprinting technologies are integrated into the AI model design.
2. Elaborate on the roles of each technology within the AI model architecture in a clear and concise manner.
3. Simplify the language used to describe the AI model architecture and algorithms for a non-technical audience to easily understand.
